In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# 일단 THGNN을 기반으로 따라가자.


df_close = pd.read_csv('./data_stock/close.csv', index_col = 0, thousands=',')
df_close.index = pd.to_datetime(df_close.index)
df_open = pd.read_csv('./data_stock/open.csv', index_col = 0, thousands=',')
df_open.index = pd.to_datetime(df_open.index)
df_high = pd.read_csv('./data_stock/high.csv', index_col = 0, thousands=',')
df_high.index = pd.to_datetime(df_high.index)
df_low = pd.read_csv('./data_stock/low.csv', index_col = 0, thousands=',')
df_low.index = pd.to_datetime(df_low.index)
df_volume = pd.read_csv('./data_stock/volume.csv', index_col = 0, thousands=',')
df_volume.index = pd.to_datetime(df_volume.index)

In [101]:
df_market_cap = pd.read_csv('./data_stock/market_equity_kospi.csv', index_col = 0, thousands=',')
df_market_cap.index = pd.to_datetime(df_market_cap.index)

In [102]:
# train, validation, test 에 대해서 나누자.

# technical indicator 산출을 위해 조금 더 길게 뽑는다.
train_start = pd.to_datetime('2015-11-01')

train_real_start = pd.to_datetime('2016-01-01')
train_end = pd.to_datetime('2018-12-31')

val_start = pd.to_datetime('2019-01-01')
val_end = pd.to_datetime('2019-12-31')

test_start = pd.to_datetime('2020-02-01')
test_end = pd.to_datetime('2020-12-31')

In [103]:
# generate_candidate_stocks
candidate_stocks = df_market_cap.loc[test_start,:].sort_values(ascending=False)[:70].index.to_list()

In [104]:
# 거래정지가 이전에 일어났던 경우에 대해서는 제외하자.
for cand_ in candidate_stocks:
    # check if there was a volume 0 in the past
    if (df_volume[train_real_start:test_start].loc[:,cand_].min() == 0) or (df_close[train_real_start:test_start].loc[:,cand_].isnull().sum() > 0):
        candidate_stocks.remove(cand_)
# 거래정지는 0으로 표기되거나 결측치로 표기되어 있을 것이다.-> 제거
for cand_ in candidate_stocks:
    if df_volume[train_real_start:test_start].loc[:,cand_].isnull().sum() > 0:
        candidate_stocks.remove(cand_)

In [105]:
len(candidate_stocks)

58

In [106]:
# 예측 진행할 종목을 50개로 제한하자.
finalist_stocks = candidate_stocks[:50]

In [107]:
close_df = df_close.loc[train_start:test_end, finalist_stocks]
open_df = df_open.loc[train_start:test_end, finalist_stocks]
high_df = df_high.loc[train_start:test_end, finalist_stocks]
low_df = df_low.loc[train_start:test_end, finalist_stocks]
volume_df = df_volume.loc[train_start:test_end, finalist_stocks]

In [108]:
open_df.isna().sum().sum()

0

In [109]:
stock = finalist_stocks[0]

In [110]:
close_df[stock]-close_df[stock].shift(1)

2015-11-02       NaN
2015-11-03    -250.0
2015-11-04     700.0
2015-11-05      50.0
2015-11-06   -1450.0
               ...  
2020-12-23    3500.0
2020-12-24    2000.0
2020-12-28   -2500.0
2020-12-29     500.0
2020-12-30    2500.0
Name: A000660, Length: 1269, dtype: float64

Distribution shift를, 그러면 close를 기준으로 해야할거같은데... sequence 자체를 넣어주어야 하나...


yelp는 그냥 그렇다고 하였다(코로나 때 엄청난 변화가 있었다)

In [141]:
# technical indicator 별 df를 따로 만든다-> 향후 utils에 편입
# source: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10137931


def get_moving_average(df, window):
    '''
    df : pd.DataFrame -> 여기는 데이터가 충분히 있는 df여야함.
    window : int -> 이동평균을 구할 window size
    '''
    return df.rolling(window=window).mean()

def moving_standard_deviation(df, window):
    '''
    df : pd.DataFrame -> 여기는 데이터가 충분히 있는 df여야함.
    window : int -> 이동평균을 구할 window size
    '''
    return df.rolling(window=window).std()

def exponential_moving_average(df, window):
    '''
    df : pd.DataFrame -> 여기는 데이터가 충분히 있는 df여야함.
    window : int -> 이동평균을 구할 window size
    '''
    return df.ewm(span=window, adjust=False).mean()

def displaced_moving_average(df, window):
    '''
    df : pd.DataFrame -> 여기는 데이터가 충분히 있는 df여야함.
    window : int -> 이동평균을 구할 window size
    '''
    return df.shift(window)


def calculate_macd(data, short_window=12, long_window=26):
    # Calculate short-term and long-term exponential moving averages (EMAs) for each stock
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()
    
    # Calculate MACD line (difference between short-term EMA and long-term EMA) for each stock
    macd_line = short_ema - long_ema
    
    # Create a DataFrame to store MACD values for each stock
    macd_data = pd.DataFrame(macd_line)
    
    return macd_data




In [142]:
moving_average_df = get_moving_average(close_df, window = 12)

In [150]:
moving_average_df

,A000660,A035420,A005380,A051910,A068270,A012330,A028260,A051900,A005490,A006400,...,A029780,A000720,A078930,A010140,A004020,A071050,A011070,A097950,A161390,A003670
2015-11-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,116833.333333,285250.000000,188541.666667,812250.000000,331449.083333,248750.000000,128791.666667,1.592333e+06,271125.000000,559166.666667,...,34775.000000,36332.500000,37720.833333,6649.833333,39166.666667,77391.666667,167000.000000,375541.666667,39158.333333,103833.333333
2020-12-24,117083.333333,285000.000000,188583.333333,812833.333333,331255.666667,249500.000000,130041.666667,1.598833e+06,271583.333333,560000.000000,...,34866.666667,36344.583333,37758.333333,6647.416667,39254.166667,77575.000000,167958.333333,377208.333333,39341.666667,104958.333333
2020-12-28,116666.666667,284416.666667,188416.666667,811000.000000,329708.250000,250208.333333,131333.333333,1.605750e+06,271625.000000,559416.666667,...,34904.166667,36296.000000,37804.166667,6644.250000,39112.500000,77741.666667,169375.000000,378833.333333,39395.833333,104791.666667
2020-12-29,116625.000000,284166.666667,188333.333333,810666.666667,330258.000000,250833.333333,132291.666667,1.613083e+06,271458.333333,562833.333333,...,34808.333333,36125.666667,37775.000000,6644.250000,39020.833333,77833.333333,170583.333333,380416.666667,39470.833333,104708.333333


In [143]:
moving_standard_df = moving_standard_deviation(close_df, window = 12)

In [144]:
ewma_df = exponential_moving_average(close_df, window = 12)

In [145]:
disp_df = displaced_moving_average(close_df, window = 12)

In [146]:
macd_df = calculate_macd(close_df, short_window=12, long_window=26)

In [152]:
for df in dfs:
    print(len(df.columns))

50
50
50
50
50
50
50
50
50
50


In [158]:
dfs = []
for df, name in zip([open_df, high_df, low_df, close_df, volume_df, moving_average_df, moving_standard_df, ewma_df, disp_df, macd_df],
                    ['open','high', 'low','close','volume','ma', 'std', 'ewma', 'disp', 'macd']):
    melted = df.reset_index().melt(id_vars='index', var_name='stock', value_name=name)
    melted.rename(columns={'index': 'dt'}, inplace=True)
    dfs.append(melted)

# 모든 데이터프레임을 하나로 결합
from functools import reduce
final_df = reduce(lambda left, right: pd.merge(left, right, on=['dt', 'stock'], how='outer'), dfs)


In [159]:
final_df

,dt,stock,open,high,low,close,volume,ma,std,ewma,disp,macd
0,2015-11-02,A000660,30700.0,32250.0,30400.0,32150.0,4493014.0,NaN,NaN,32150.000000,NaN,0.000000
1,2015-11-03,A000660,32500.0,33000.0,31550.0,31900.0,4453120.0,NaN,NaN,32111.538462,NaN,-19.943020
2,2015-11-04,A000660,32500.0,32700.0,32100.0,32600.0,3276995.0,NaN,NaN,32186.686391,NaN,20.499834
3,2015-11-05,A000660,32350.0,32700.0,32200.0,32650.0,1928001.0,NaN,NaN,32257.965407,NaN,55.940818
4,2015-11-06,A000660,32350.0,32400.0,31050.0,31200.0,5219855.0,NaN,NaN,32095.201499,NaN,-32.599048
...,...,...,...,...,...,...,...,...,...,...,...,...
63445,2020-12-23,A003670,107000.0,108000.0,102000.0,104500.0,487442.0,103833.333333,4381.434662,102330.256168,94800.0,6799.775293
63446,2020-12-24,A003670,104500.0,106000.0,103500.0,104500.0,381566.0,104958.333333,1698.372661,102664.062911,91000.0,6469.173212
63447,2020-12-28,A003670,104500.0,105500.0,102000.0,103500.0,535222.0,104791.666667,1738.054468,102792.668617,105500.0,6056.659636
63448,2020-12-29,A003670,104500.0,105500.0,102500.0,105000.0,482340.0,104708.333333,1698.372661,103132.258061,106000.0,5784.101597


In [160]:
#save it as pickle file
final_df.to_pickle('./data_stock/final_df.pkl')

In [161]:
date_list=final_df['dt'].unique()


In [ ]:
from tqdm import tqdm
feature_cols = final_df.columns.tolist()[2:]
#dt is the last trading day of each month
dt=[train_start,'2022-12-30']

for i in range(len(dt)):
    df2 = df1.copy()
    end_data = dt[i]
    start_data = stock_trade_data[stock_trade_data.index(end_data)-(prev_date_num - 1)]
    df2 = df2.loc[df2['dt'] <= end_data]
    df2 = df2.loc[df2['dt'] >= start_data]
    code = sorted(list(set(df2['code'].values.tolist())))
    test_tmp = {}
    for j in tqdm(range(len(code))):
        df3 = df2.loc[df2['code'] == code[j]]
        y = df3[feature_cols].values
        if y.T.shape[1] == prev_date_num:
            test_tmp[code[j]] = y.T

In [ ]:
import time
import pickle
import multiprocessing as mp
import numpy as np
import pandas as pd
from tqdm import tqdm

feature_cols = ['high','low','close','open','to','vol']

def cal_pccs(x, y, n):
    sum_xy = np.sum(np.sum(x*y)) #요소간 곱한 것을 모두 더하기 위함임. e.g. [1,2,3]*[1,2,3] -> 근데 굳이 이렇게 해야하나...
    sum_x = np.sum(np.sum(x))
    sum_y = np.sum(np.sum(y))
    sum_x2 = np.sum(np.sum(x*x))
    sum_y2 = np.sum(np.sum(y*y))
    pcc = (n*sum_xy-sum_x*sum_y)/np.sqrt((n*sum_x2-sum_x*sum_x)*(n*sum_y2-sum_y*sum_y))
    return pcc

def calculate_pccs(xs, yss, n): # ref_dict[code], ref_dict, n
    '''
    각 stock code의 각 feature 별로 상관관계가 계산된다. 이 때 tmp_res에는 각 주식 code별 상관관계가 저장되며 임시 tmp_res는 results에 저장된다.
     '''
    result = []
    for name in yss:
        ys = yss[name]
        tmp_res = []
        for pos, x in enumerate(xs): # pos는 feature의 index, x는 feature의 20일 동안의 데이터
            y = ys[pos] # 하나의 feature에 대해서 20일 어치의 데이터를 가져온다.
            tmp_res.append(cal_pccs(x, y, n)) # x: 주식 code, y: features, n: 참조할 과거 시계열 길이 -> pearson 상관관계 값을 구한다.
        result.append(tmp_res) # 각code 별로 진행.
    return np.mean(result, axis=1) # mean of all pccs -> 6개의 feature에 대한 평균 pccs -> 이렇게 하면 안되지 않니... 어떻게 했지 

def stock_cor_matrix(ref_dict, codes, n, processes=1):
    if processes > 1:
        pool = mp.Pool(processes=processes)
        args_all = [(ref_dict[code], ref_dict, n) for code in codes] # 전체 코드에 대해서 각각의 코드에 대한 pccs를 구하고 튜플로서 저장
        results = [pool.apply_async(calculate_pccs, args=args) for args in args_all] # 저장된 튜플(x,y,n)이 있으면 그것에 대해서 멀
        output = [o.get() for o in results]
        data = np.stack(output) #  하나의 배열로서 합친다.
        return pd.DataFrame(data=data, index=codes, columns=codes)
    data = np.zeros([len(codes), len(codes)])
    for i in tqdm(range(len(codes))): # 행은 code, 열은 feature
        data[i, :] = calculate_pccs(ref_dict[codes[i]], ref_dict, n)
    return pd.DataFrame(data=data, index=codes, columns=codes)

path1 = "./data/csi300.pkl"
df1 = pickle.load(open(path1, 'rb'), encoding='utf-8')
#prev_date_num Indicates the number of days in which stock correlation is calculated
prev_date_num = 20
date_unique=df1['dt'].unique()
stock_trade_data=date_unique.tolist()
stock_trade_data.sort()
stock_num=df1.code.unique().shape[0]
#dt is the last trading day of each month
dt=['2022-11-30','2022-12-30']

df1['dt']=df1['dt'].astype('datetime64')

for i in range(len(dt)): # datetime list가 생성되었는데, 이는 각 달의 마지막 날짜를 의미한다.
    df2 = df1.copy()
    end_data = dt[i]
    start_data = stock_trade_data[stock_trade_data.index(end_data)-(prev_date_num - 1)]
    df2 = df2.loc[df2['dt'] <= end_data]
    df2 = df2.loc[df2['dt'] >= start_data]
    code = sorted(list(set(df2['code'].values.tolist())))
    test_tmp = {}
    for j in tqdm(range(len(code))):
        df3 = df2.loc[df2['code'] == code[j]]
        y = df3[feature_cols].values
        if y.T.shape[1] == prev_date_num:
            test_tmp[code[j]] = y.T
    t1 = time.time()
    result = stock_cor_matrix(test_tmp, list(test_tmp.keys()), prev_date_num, processes=1) 
    # test_tmp: {code: feature} -> code에 대한 feature를 가지고 있는 dict
    # test_tmp.keys(): code list
    # prev_date_num: 20
    # processes: 1 -> 멀티프로세싱을 사용하지 않는다. 차후 멀티프로세싱 사용 시 변경
    result=result.fillna(0)
    for i in range(0,stock_num):
        result.iloc[i,i]=1 # 대각선은 1로 채워준다. -> Adjacency matrix이기 때문에 대각선은 1로 채워주는 작업 assert
    t2 = time.time()
    print('time cost', t2 - t1, 's')
    result.to_csv("./data/relation/"+str(end_data)+".csv")
    
    '''
    그냥 데이터 가지고 
    '''
